In [ ]:
%%spark
--conf spark.yarn.queue=root.zw02.hadoop-wallepnc.query
--conf spark.driver.maxResultSize=4g
--conf spark.driver.memory=8g
--conf spark.executor.memory=8g
--conf spark.executor.cores=2
--conf spark.yarn.executor.memoryOverhead=2072
--conf spark.dynamicAllocation.enabled=true
--conf spark.dynamicAllocation.minExecutors=600
--conf spark.dynamicAllocation.maxExecutors=1000
--conf spark.sql.shuffle.partitions=6000
--conf spark.memory.fraction=0.7
--conf spark.sql.autoBroadcastJoinThreshold=1048576000

In [ ]:
db = 'mart_wallepnc_hulk'

In [ ]:
%%sql r --preview --quiet
SELECT DISTINCT get_json_object(`debug`,'$.planningData.obstacleDecider.decisions[0].type') as obstacles_decision,
       get_json_object(`debug`,'$.planningData.obstacleDecider.decisions[0].id') as obstacles_id,
       autocar_name,
       timestamp,
       hour,
       `git-branch`,
       main_decision,
       record_name,
        is_auto
  FROM mart_wallepnc_hulk.detail_planning
 where dt='20210114'
   and autocar_name = 'zcx-04'
   and frame_id % 10 = 0
   and get_json_object(`debug`,'$.planningData.obstacleDecider.decisions[0].type') not like 'NO_NUDGE'
   and get_json_object(`debug`,'$.planningData.obstacleDecider.decisions[0].sMin') < get_json_object(`debug`,'$.planningData.piecewiseJerkPath[0].pathPoints[0].s')
   and get_json_object(`debug`,'$.planningData.obstacleDecider.decisions[0].sMax') > get_json_object(`debug`,'$.planningData.piecewiseJerkPath[0].pathPoints[0].s')
   and get_json_object(`debug`,'$.planningData.obstacleDecider.decisions[0]') is not null


In [ ]:
%%sql p --preview --quiet

create table if not exists {db}.bdecision_decision
(
    obstacles_decision string comment 'obstacle decision',
    obstacles_id string comment 'obstacles_id',
    autocar_name string COMMENT '主车的名字,例：mkz-00',
    `timestamp` double COMMENT '时间戳,UNIXtime,单位为s',
    `hour` string COMMENT '两位数24小时制表示小时',
    `git-branch` string COMMENT 'git branch name',
     `main_decision` string COMMENT '自车决策：cruise、stop、estop、change_lane、mission_complete、not_ready、parking',
    record_name string COMMENT "record_name",
     `is_auto` string COMMENT '是否处于自动驾驶模式,字符串true或false'
) comment 'bdecision_environment Demo表' stored as orc

In [ ]:
r.createOrReplaceTempView('my_tmp_view')

In [ ]:
%%sql p --preview --quiet

insert into {db}.bdecision_decision
select * from my_tmp_view

In [ ]:
%%sql p --preview --quiet
select * from {db}.bdecision_decision